In [1]:
import os
import openai
import dotenv
from tqdm import tqdm
from datasets import load_dataset

dotenv.load_dotenv()

True

In [3]:
client = openai.OpenAI()

system = {
    "role": "system",
    "content": """
You are senior database engineer, you are very experienced in database design and optimization.
Your coworker ask you to help him create SQL query to get the data from the database.

If user describe the problem, you answer the question with SQL.
Give Them the SQL query to solve the problem.

For example:
- "I want get user, who salary is more than 1000 and work in the company more than 5 years.": "SELECT * FROM users WHERE salary > 1000 AND work_years > 5;"
- "Who to get user who dept is 'IT' or 'Engineering'": "SELECT * FROM users u LEFT JOIN departments d ON u.dept_id = d.dept_id WHERE d.dept_name = 'IT' OR d.dept_name = 'Engineering';"
- "I want to get the user who has boss name is 'John' but recursively": "WITH RECURSIVE cte AS (SELECT * FROM users WHERE boss_name = 'John' UNION ALL SELECT u.* FROM users u JOIN cte ON u.boss_id = cte.user_id) SELECT * FROM cte;"

Think step by step, and give them the SQL query to solve the problem.
""",
}

In [4]:
result = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        system,
        {
            "role": "user",
            "content": """how to get the user who has the highest salary in the company? order by salary and top 5 please""",
        },
    ],
)
print(result.choices[0].message.content)

To get the users with the highest salaries in the company, ordered by salary in descending order and limited to the top 5, you can use the following SQL query:

```sql
SELECT * FROM users ORDER BY salary DESC LIMIT 5;
```


In [8]:
result = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        system,
        {
            "role": "user",
            "content": """get all user who boss name is 'Ann' but recursively only 2 depth, there is no depth value so SQL must count there depth""",
        },
    ],
)
print(result.choices[0].message.content)
# 예상처럼 동작을 안함...

To retrieve all users whose boss's name is 'Ann' with a recursive query limited to 2 depths, you can use the following SQL query:

```sql
WITH RECURSIVE cte AS (
    -- Base case: Select users whose boss name is 'Ann'
    SELECT * FROM users WHERE boss_name = 'Ann'
    
    UNION ALL
    
    -- Recursive case: Select users whose boss is in the first level of results (depth 1)
    SELECT u.* FROM users u
    JOIN cte ON u.boss_id = cte.user_id
)
SELECT * FROM cte WHERE depth <= 2; -- Ensure we limit the depth to 2
```

This query starts by selecting users with 'Ann' as their boss. Then, it recursively selects users under those users while counting the depth. It restricts the result to a depth of 2 levels. Please note that you may need to implement the depth counting in the recursive CTE, depending on your database system's capabilities.
